In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [51]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.optim import Adam, SGD
import numpy as np
import time
import matplotlib.pyplot as plt
from timeit import default_timer as timer

In [54]:
## Clean Torch implementation ##

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        # Defining the model layer
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    # Definition of the forward model path
    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        return x

# Calculation of cross-entropy loss
def cost_function(output, target):
    loss = F.cross_entropy(output, target)
    return loss

def predict(model, X):
    with torch.no_grad():
        # forward path
        output = model(X)
        _, predicted = torch.max(output, 1)
        return predicted

def main():
    # Load the training and test datasets
    train = np.genfromtxt('gdrive/MyDrive/train.csv', delimiter=',')
    test = np.genfromtxt('gdrive/MyDrive/test.csv', delimiter=',')

    train_label = train[:,0].astype(int)
    test_label = test[:,0].astype(int)

    # normalization
    train = torch.tensor(train[:,1:] / 255., dtype=torch.float32)
    test = torch.tensor(test[:,1:] / 255., dtype=torch.float32)

    X = train
    y = torch.tensor(train_label, dtype=torch.long)

    input_size = X.shape[1]
    hidden_size = 8
    output_size = 3

    model = NeuralNetwork(input_size, hidden_size, output_size)
    model.cuda()  # Move model to GPU


    # Create the optimizer
    optimizer = Adam(model.parameters(), lr=0.001)

    for epoch in range(600):  # Training for 10 epochs
        optimizer.zero_grad()
        output = model(X.cuda())  # Move input to GPU
        loss = cost_function(output, y.cuda())  # Move target to GPU
        loss.backward()
        optimizer.step()

    train_pred = predict(model, train.cuda())  # Move input to GPU
    test_pred = predict(model, test.cuda())  # Move input to GPU

    print('accuracy on training set =', torch.sum(train_pred.cpu() == y).item() / len(train_label))
    print('accuracy on test set =', torch.sum(test_pred.cpu() == torch.tensor(test_label)).item() / len(test_label))

In [56]:
# Torch implementation of the original neural network sturcture
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        # create torch network levels
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        # g(x) sigmoid activation function
        self.sigmoid = nn.Sigmoid()

    # Definition of the forward path
    def forward(self, x):
        out = self.fc1(x)
        out = self.sigmoid(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

# Loss function with added regularization term
def cost_function(predictions, targets, criterion, model, lmbda=0):
    loss = criterion(predictions, targets)
    reg_loss = 0
    for param in model.parameters():
        reg_loss += torch.sum(torch.square(param))
    loss += (lmbda / 2) * reg_loss
    return loss

def predict(model, X):
    with torch.no_grad():
        # forward path
        outputs = model(X)
        _, predicted = torch.max(outputs, 1)
    return predicted

def main():
    np.random.seed(917)

    # Load the training and test datasets
    train = np.genfromtxt('gdrive/MyDrive/train.csv', delimiter=',')
    test = np.genfromtxt('gdrive/MyDrive/test.csv', delimiter=',')

    # get labels (0=Elliptical, 1=Spiral, 2=Irregular)
    train_label = train[:,0].reshape(len(train),1)
    test_label = test[:,0].reshape(len(test),1)

    # normalize image data to [0,1]
    train = train[:,1:] / 255.
    test = test[:,1:] / 255.

    # Convert numpy arrays to PyTorch tensors
    X_train = torch.tensor(train, dtype=torch.float32)
    y_train = torch.tensor(train_label, dtype=torch.long)
    X_test = torch.tensor(test, dtype=torch.float32)
    y_test = torch.tensor(test_label, dtype=torch.long)

    input_size = X_train.shape[1]
    hidden_size = 8
    num_classes = 3
    learning_rate = 0.1
    num_epochs = 600
    lmbda = 0.0002

    model = NeuralNetwork(input_size, hidden_size, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=0.001)

    Js_train = []
    Js_test = []

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = cost_function(outputs, y_train.squeeze(), criterion, model, lmbda)
        loss.backward()
        optimizer.step()

        model.eval()
        train_pred = predict(model, X_train)
        train_acc = torch.sum(train_pred == y_train.squeeze()).item() / len(y_train)
        Js_train.append(loss.item())

        test_pred = predict(model, X_test)
        test_acc = torch.sum(test_pred == y_test.squeeze()).item() / len(y_test)
        test_loss = cost_function(model(X_test), y_test.squeeze(), criterion, model, lmbda).item()
        Js_test.append(test_loss)

    # print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Train Acc: {train_acc:.2f}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}")
    print('accuracy on training set =', train_acc)
    print('accuracy on test set =', test_acc)

    # plt.figure(figsize=(10, 5))
    # plt.subplot(1, 2, 1)
    # plt.plot(Js_train, label='Train')
    # plt.xlabel('Epoch')
    # plt.ylabel('Loss')
    # plt.title('Training Loss')
    # plt.legend()

    # plt.subplot(1, 2, 2)
    # plt.plot(Js_test, label='Test')
    # plt.xlabel('Epoch')
    # plt.ylabel('Loss')
    # plt.title('Test Loss')
    # plt.legend()

    # plt.show()

In [57]:
%%time
main()

accuracy on training set = 0.5303703703703704
accuracy on test set = 0.5525925925925926
CPU times: user 7.42 s, sys: 70.3 ms, total: 7.49 s
Wall time: 7.58 s


In [58]:
iterations = 10
total_time = 0

for _ in range(iterations):
    start_time = time.perf_counter()  # or time.time()
    main()
    end_time = time.perf_counter()  # or time.time()
    total_time += (end_time - start_time)

average_time = total_time / iterations
print(f"Average execution time: {average_time} seconds")

accuracy on training set = 0.5774074074074074
accuracy on test set = 0.5881481481481482
accuracy on training set = 0.6048148148148148
accuracy on test set = 0.6118518518518519
accuracy on training set = 0.5955555555555555
accuracy on test set = 0.5955555555555555
accuracy on training set = 0.6559259259259259
accuracy on test set = 0.6503703703703704
accuracy on training set = 0.5396296296296297
accuracy on test set = 0.562962962962963
accuracy on training set = 0.5766666666666667
accuracy on test set = 0.5866666666666667
accuracy on training set = 0.5651851851851852
accuracy on test set = 0.562962962962963
accuracy on training set = 0.5637037037037037
accuracy on test set = 0.5703703703703704
accuracy on training set = 0.6914814814814815
accuracy on test set = 0.6903703703703704
accuracy on training set = 0.6037037037037037
accuracy on test set = 0.6192592592592593
Average execution time: 7.038278227700085 seconds
